In [1]:
import copy

import openmm
from openff.toolkit.topology.molecule import Molecule, unit
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem
import MDAnalysis as mda

/lila/data/chodera/lilywang/micromamba/envs/fb-193-tk-010-oe-2022/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/lila/data/chodera/lilywang/micromamba/envs/fb-193-tk-010-oe-2022/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/lila/data/chodera/lilywang/micromamba/envs/fb-193-tk-010-oe-2022/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/lila/data/chodera/lilywang/micromamba/envs/fb-193-tk-010-oe-2022/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest su

In [2]:
dir1 = "fb-193-tk-010-oe-2022/rep4"
dir2 = "fb-195-tk-013-oe-2022-interchange-replace-cache-switching/rep1"

In [3]:
batch = "opt-geo-batch-113"
system = "19095588-8"

In [4]:
qm_sdf = f"{dir1}/targets/{batch}/{system}.sdf"
mol = Molecule.from_file(qm_sdf, "SDF", allow_undefined_stereo=True)
openmm_topology = mol.to_topology().to_openmm()

In [5]:
u_qm = mda.Universe(mol.to_rdkit())
u_qm.atoms.write("qm_structure.xyz")

In [6]:
xml1 = f"{dir1}/optimize.tmp/{batch}/iter_0000/{system}_mminit.xml"
xml2 = f"{dir2}/optimize.tmp/{batch}/iter_0000/{system}_mminit.xml"

u1 = mda.Universe(f"{dir1}/optimize.tmp/{batch}/iter_0000/{system}_mmopt.xyz")
xyz1 = u1.atoms.positions
u2 = mda.Universe(f"{dir2}/optimize.tmp/{batch}/iter_0000/{system}_mmopt.xyz")
xyz2 = u2.atoms.positions

In [7]:
u1.atoms.write("fb_193.xyz")
u2.atoms.write("fb_195.xyz")

In [8]:
with open(xml1) as f:
     sys1 = openmm.XmlSerializer.deserialize(f.read())

with open(xml2) as f:
     sys2 = openmm.XmlSerializer.deserialize(f.read())

In [9]:
def get_energy(system, minimize: bool = True):
    integrator = openmm.LangevinIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        1 * openmm.unit.femtosecond
    )
    
    force_group_order = {
        0: "NonbondedForce",
        1: "PeriodicTorsionForce",
        2: "HarmonicBondForce",
        3: "HarmonicAngleForce",
    }
    inverse = {v: k for k, v in force_group_order.items()}
    for force in system.getForces():
        group = inverse[force.__class__.__name__]
        force.setForceGroup(group)

    platform = openmm.Platform.getPlatformByName("Reference")

    sim1 = openmm.app.Simulation(openmm_topology, system, integrator, platform)
    sim1.context.setPositions(mol.conformers[0])
    if minimize:
        crit = 1e-4
        steps = int(max(1, -1*np.log10(crit)))
        kj_per_mol = openmm.unit.kilojoule / openmm.unit.mole
        
        for logc in np.linspace(0, np.log10(crit), steps):
            sim1.minimizeEnergy(tolerance=10**logc*kj_per_mol, maxIterations=100000)
        
        for _ in range(1000):
            e_minimized = sim1.context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(kj_per_mol)
            sim1.minimizeEnergy(tolerance=crit*kj_per_mol, maxIterations=10)
            e_new = sim1.context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(kj_per_mol)
            if abs(e_new - e_minimized) < crit * 10:
                break
            
    e1 = sim1.context.getState(getEnergy=True).getPotentialEnergy()
    pos = sim1.context.getState(getPositions=True)
    angstrom = pos.getPositions(asNumpy=True).value_in_unit(openmm.unit.angstrom)
    
    data = {"total_energy": e1._value}
    for group, force_name in force_group_order.items():
        state = sim1.context.getState(getEnergy=True, groups={group})
        data[force_name] = state.getPotentialEnergy()._value
    return data, angstrom

In [10]:
data1, min1 = get_energy(sys1)

In [11]:
data2, min2 = get_energy(sys2)

In [12]:
data1

{'total_energy': -91.20441098709435,
 'NonbondedForce': -282.3557755879145,
 'PeriodicTorsionForce': 109.541982014749,
 'HarmonicBondForce': 9.166777114581903,
 'HarmonicAngleForce': 72.44260547148923}

In [13]:
data2

{'total_energy': -91.20441098707292,
 'NonbondedForce': -282.35578699014656,
 'PeriodicTorsionForce': 109.54199941713894,
 'HarmonicBondForce': 9.166776931376813,
 'HarmonicAngleForce': 72.4425996545579}

In [14]:
def print_rmsd(qm_mol, coords1, coords2):
    qm_mol = copy.deepcopy(qm_mol)
    rdmol = qm_mol.to_rdkit()
    
    coords1 = np.array(coords1).astype(float)
    qm_mol._conformers = [coords1 * unit.angstrom]
    rdmol1 = qm_mol.to_rdkit()
    a_to_qm = AllChem.GetBestRMS(rdmol, rdmol1)
    print(f"Mol 1 to QM: {a_to_qm}")
    
    coords2 = np.array(coords2).astype(float)
    qm_mol._conformers = [coords2 * unit.angstrom]
    rdmol2 = qm_mol.to_rdkit()
    b_to_qm = AllChem.GetBestRMS(rdmol, rdmol2)
    print(f"Mol 2 to QM: {b_to_qm}")
    
    a_to_b = AllChem.GetBestRMS(rdmol1, rdmol2)
    print(f"Mol 1 to Mol 2: {a_to_b}")

In [15]:
print_rmsd(mol, xyz1, xyz2)

Mol 1 to QM: 2.97829864887902
Mol 2 to QM: 1.8349996744698331
Mol 1 to Mol 2: 2.18062317618241


In [16]:
print_rmsd(mol, xyz1, min1)

Mol 1 to QM: 2.97829864887902
Mol 2 to QM: 1.8349994219228138
Mol 1 to Mol 2: 2.18062312190854


In [17]:
print_rmsd(mol, xyz2, min1)

Mol 1 to QM: 1.8349996744698271
Mol 2 to QM: 1.8349994219228138
Mol 1 to Mol 2: 2.4040990281336426e-06


In [18]:
print_rmsd(mol, min1, min2)

Mol 1 to QM: 1.8349994219228318
Mol 2 to QM: 1.8349992167261757
Mol 1 to Mol 2: 1.270283288159683e-06


In [64]:
fcs1 = {
    x.__class__.__name__: x
    for x in sys1.getForces()
}
fcs2 = {
    x.__class__.__name__: x
    for x in sys2.getForces()
}

In [65]:
fcs1

{'PeriodicTorsionForce': <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2af2e31b5050> >,
 'NonbondedForce': <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2af2e31b4b40> >,
 'HarmonicBondForce': <openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2af2e31b4570> >,
 'HarmonicAngleForce': <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2af2e31b43c0> >}

In [66]:
tors1 = fcs1["PeriodicTorsionForce"]
tors2 = fcs2["PeriodicTorsionForce"]

for i in range(tors1.getNumTorsions()):
    p1_ = tors1.getTorsionParameters(i)
    p1 = np.array(p1_[:5] + [x._value for x in p1_[5:]])
    p2_ = tors2.getTorsionParameters(i)
    p2 = np.array(p2_[:5] + [x._value for x in p2_[5:]])
    
    assert np.allclose(p1, p2)

In [68]:
nb1 = fcs1["NonbondedForce"]
nb2 = fcs2["NonbondedForce"]
for i in range(nb1.getNumParticles()):
    p1 = np.array([
        x._value for x in nb1.getParticleParameters(i)
    ])
    p2 = np.array([
        x._value for x in nb2.getParticleParameters(i)
    ])
    assert np.allclose(p1, p2)

In [69]:
nb1.getSwitchingDistance()

Quantity(value=-1.0, unit=nanometer)

In [70]:
nb2.getSwitchingDistance()

Quantity(value=-1.0, unit=nanometer)